In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import json

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
FEATURES_FILE = "/content/drive/MyDrive/Edulift/eVED_features.csv"
MATCHED_FILE  = "/content/drive/MyDrive/Edulift/eVED_171101_week_matched.csv"   # optional (for labeling)
REPORT_DIR    = "/content/drive/MyDrive/Edulift/reports/t6"

In [4]:
import os

os.makedirs(REPORT_DIR, exist_ok=True)

In [5]:
def save_hist(series, bins, xlabel, title, outfile):
    s = pd.to_numeric(series, errors="coerce").dropna()
    plt.figure(figsize=(9,5))
    plt.hist(s, bins=bins)
    plt.title(title)
    plt.xlabel(xlabel)
    plt.ylabel("Count")
    plt.tight_layout()
    plt.savefig(outfile, dpi=150)
    plt.close()

In [6]:
def looks_like_real_clock(dt_series: pd.Series) -> bool:
    """Return True if timestamps span multiple days or hours beyond 0–1."""
    try:
        d = pd.to_datetime(dt_series, errors="coerce")
    except Exception:
        return False
    if d.isna().all():
        return False
    # If all dates are the same (e.g., 1970-01-01) and hours all 0, it's not real clock
    # If we see multiple calendar days or a wider hour spread, treat as real clock
    days = d.dt.normalize().nunique()
    hour_nunique = d.dt.hour.nunique()
    # Heuristics
    return (days > 1) or (hour_nunique > 3)

In [7]:
def try_build_route_labels(features_df: pd.DataFrame, matched_path: str):
    """
    Optional: add human-friendly route labels by taking first/last road_name per Trip
    if the matched points file is available. Falls back silently if not.
    """
    if not os.path.exists(matched_path):
        return features_df

    try:
        m = pd.read_csv(matched_path, usecols=["Trip","timestamp","road_name",
                                               "matched_lat","matched_lon"])
    except Exception:
        return features_df

    # First/last road_name per Trip
    m["timestamp"] = pd.to_datetime(m["timestamp"], errors="coerce")
    g = m.sort_values(["Trip","timestamp"]).groupby("Trip")
    start_names = g.first()["road_name"].rename("start_road_name")
    end_names   = g.last()["road_name"].rename("end_road_name")

    out = features_df.merge(start_names, left_on="Trip", right_index=True, how="left")
    out = out.merge(end_names, left_on="Trip", right_index=True, how="left")

    # Make a compact route label
    def route_label(r):
        a = r.get("start_road_name")
        b = r.get("end_road_name")
        if pd.isna(a) and pd.isna(b):
            return f"OD {int(r['od_cluster_start'])}->{int(r['od_cluster_end'])}" \
                   if pd.notna(r["od_cluster_start"]) and pd.notna(r["od_cluster_end"]) else "Unknown"
        a = a if pd.notna(a) else "?"
        b = b if pd.notna(b) else "?"
        return f"{a} → {b}"

    out["route_label"] = out.apply(route_label, axis=1)
    return out

In [8]:
df = pd.read_csv(FEATURES_FILE)

In [9]:
df.head()

,Trip,start_time,end_time,duration_min,distance_km,avg_speed_kmh,matched_lat_start,matched_lon_start,od_cluster_start,matched_lat_end,matched_lon_end,od_cluster_end
0,8.0,1970-01-01,1970-01-01 00:07:05.300,7.088333,4.204831,35.592264,42.281343,-83.735720,0,42.262099,-83.704433,22
1,24.0,1970-01-01,1970-01-01 00:10:23.500,10.391667,6.370776,36.783952,42.256714,-83.695647,1,42.281597,-83.753555,0
2,27.0,1970-01-01,1970-01-01 00:08:03.000,8.050000,5.219052,38.899767,42.271846,-83.727940,0,42.252823,-83.674770,8
3,32.0,1970-01-01,1970-01-01 00:03:42.800,3.713333,1.917766,30.987236,42.284952,-83.801928,2,42.290112,-83.793830,-1
4,34.0,1970-01-01,1970-01-01 00:13:00.400,13.006667,7.093905,32.724317,42.274646,-83.674135,3,42.282448,-83.745979,0


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 655 entries, 0 to 654
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Trip               655 non-null    float64
 1   start_time         655 non-null    object 
 2   end_time           655 non-null    object 
 3   duration_min       655 non-null    float64
 4   distance_km        653 non-null    float64
 5   avg_speed_kmh      653 non-null    float64
 6   matched_lat_start  655 non-null    float64
 7   matched_lon_start  655 non-null    float64
 8   od_cluster_start   655 non-null    int64  
 9   matched_lat_end    655 non-null    float64
 10  matched_lon_end    655 non-null    float64
 11  od_cluster_end     655 non-null    int64  
dtypes: float64(8), int64(2), object(2)
memory usage: 61.5+ KB


In [11]:
df["start_time"] = pd.to_datetime(df["start_time"], errors="coerce")
df["end_time"]   = pd.to_datetime(df["end_time"], errors="coerce")

In [12]:
 df = try_build_route_labels(df, MATCHED_FILE)

In [13]:
df.head()

,Trip,start_time,end_time,duration_min,distance_km,avg_speed_kmh,matched_lat_start,matched_lon_start,od_cluster_start,matched_lat_end,matched_lon_end,od_cluster_end,start_road_name,end_road_name,route_label
0,8.0,1970-01-01,1970-01-01 00:07:05.300,7.088333,4.204831,35.592264,42.281343,-83.735720,0,42.262099,-83.704433,22,"East Huron Street, I 94 BL, US 23 BUS",Overridge Drive,"East Huron Street, I 94 BL, US 23 BUS → Overri..."
1,24.0,1970-01-01,1970-01-01 00:10:23.500,10.391667,6.370776,36.783952,42.256714,-83.695647,1,42.281597,-83.753555,0,"Washtenaw Avenue, I 94 BL, US 23 BUS","West Huron Street, I 94 BL","Washtenaw Avenue, I 94 BL, US 23 BUS → West Hu..."
2,27.0,1970-01-01,1970-01-01 00:08:03.000,8.050000,5.219052,38.899767,42.271846,-83.727940,0,42.252823,-83.674770,8,Baldwin Avenue,"Washtenaw Avenue, M 17","Baldwin Avenue → Washtenaw Avenue, M 17"
3,32.0,1970-01-01,1970-01-01 00:03:42.800,3.713333,1.917766,30.987236,42.284952,-83.801928,2,42.290112,-83.793830,-1,Jackson Road,Dexter Road,Jackson Road → Dexter Road
4,34.0,1970-01-01,1970-01-01 00:13:00.400,13.006667,7.093905,32.724317,42.274646,-83.674135,3,42.282448,-83.745979,0,Geddes Road,North Fifth Avenue,Geddes Road → North Fifth Avenue


IndentationError: unexpected indent (ipython-input-114999730.py, line 2)

In [15]:
peak_report = {}
if looks_like_real_clock(df["start_time"]):
    df["hour"] = df["start_time"].dt.hour
    # Save histogram by hour-of-day
    save_hist(df["hour"], bins=range(0,25),
              xlabel="Hour of day", title="Trip Starts by Hour of Day",
              outfile=os.path.join(REPORT_DIR, "peak_times_by_hour.png"))
    # Weekday distribution
    df["weekday"] = df["start_time"].dt.day_name()
    weekday_counts = df["weekday"].value_counts().reindex(
        ["Monday","Tuesday","Wednesday","Thursday","Friday","Saturday","Sunday"]
    )
    weekday_counts.to_csv(os.path.join(REPORT_DIR, "trips_by_weekday.csv"))
    peak_report["mode"] = "real_clock"
    peak_report["note"] = "Computed by hour-of-day and weekday."
else:
    # Fallback: use duration buckets as 'peak usage windows' proxy
    bins = [0,5,10,15,20,30,45,60,90,120,9999]
    labels = ["0-5","5-10","10-15","15-20","20-30","30-45","45-60","60-90","90-120","120+"]
    dur = pd.to_numeric(df["duration_min"], errors="coerce").fillna(0)
    df["duration_bucket_min"] = pd.cut(dur, bins=bins, labels=labels, right=False)
    counts = df["duration_bucket_min"].value_counts().sort_index()
    counts.to_csv(os.path.join(REPORT_DIR, "peak_duration_buckets.csv"))
    save_hist(dur, bins=30,
              xlabel="Duration (min)", title="Trip Duration Distribution (Fallback)",
              outfile=os.path.join(REPORT_DIR, "duration_distribution_fallback.png"))
    peak_report["mode"] = "fallback_duration"
    peak_report["note"] = "No real clock detected; used duration buckets as peak proxy."

In [16]:
routes = (
    df.groupby(["od_cluster_start","od_cluster_end"])
      .size()
      .reset_index(name="count")
      .sort_values("count", ascending=False)
)
# If route_label exists, provide a friendly view
if "route_label" in df.columns:
    rl = (df.groupby(["od_cluster_start","od_cluster_end","route_label"])
            .size().reset_index(name="count")
            .sort_values("count", ascending=False))
    rl.to_csv(os.path.join(REPORT_DIR, "top_routes_labeled.csv"), index=False)
routes.to_csv(os.path.join(REPORT_DIR, "top_routes.csv"), index=False)

In [17]:
save_hist(df["duration_min"],  bins=40, xlabel="Minutes",
              title="Trip Duration Distribution", outfile=os.path.join(REPORT_DIR, "duration_hist.png"))
save_hist(df["distance_km"],   bins=40, xlabel="Kilometers",
          title="Trip Distance Distribution", outfile=os.path.join(REPORT_DIR, "distance_hist.png"))
save_hist(df["avg_speed_kmh"], bins=40, xlabel="km/h",
          title="Average Trip Speed Distribution", outfile=os.path.join(REPORT_DIR, "speed_hist.png"))

In [18]:
fuel_l_per_km = 8.9 / 100.0
co2_per_liter = 2.31
dist_km = pd.to_numeric(df["distance_km"], errors="coerce").fillna(0)
est_fuel_l = (dist_km * fuel_l_per_km).sum()
est_co2_kg = est_fuel_l * co2_per_liter

In [20]:
summary = {
    "trips_total": int(df.shape[0]),
    "routes_unique": int(routes.shape[0]),
    "distance_total_km": float(dist_km.sum()),
    "duration_total_hours": float(pd.to_numeric(df["duration_min"], errors="coerce").fillna(0).sum()/60.0),
    "avg_speed_kmh_median": float(pd.to_numeric(df["avg_speed_kmh"], errors="coerce").median()),
    "carbon_baseline": {
        "assumption_l_per_100km": 8.9,
        "emission_factor_kg_per_l": 2.31,
        "fuel_est_total_liters": float(est_fuel_l),
        "co2_est_total_kg": float(est_co2_kg)
    },
    "peak_times": peak_report
}
with open(os.path.join(REPORT_DIR, "summary.json"), "w") as f:
    json.dump(summary, f, indent=2)

In [21]:
    print("✅ T6 EDA complete.")
    print(f"• Wrote histograms & CSVs to: {REPORT_DIR}")
    print(f"• Summary: {os.path.join(REPORT_DIR, 'summary.json')}")
    if peak_report["mode"] == "fallback_duration":
        print("ℹ️ Used duration buckets for 'peak' analysis (no real clock in timestamps).")

✅ T6 EDA complete.
• Wrote histograms & CSVs to: /content/drive/MyDrive/Edulift/reports/t6
• Summary: /content/drive/MyDrive/Edulift/reports/t6/summary.json
ℹ️ Used duration buckets for 'peak' analysis (no real clock in timestamps).
